# Sentiment Analysis

In [ ]:
# import necessary packages
from transformers import pipeline, AutoTokenizer
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

Device set to use cpu
C:\Users\erina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
file_path = Path("data/processed")
politician = []
type_of_speech = []
file_name= []
text = []

for file in file_path.rglob("*.txt"):
    separated_file_name = str(file).split("\\") # windows

    politician.append(separated_file_name[2])
    type_of_speech.append(separated_file_name[3])
    file_name.append(separated_file_name[4])
    with open(file, 'r', encoding='utf-8') as f:
        content = f.read()
        text.append(content)

df = pd.DataFrame({
    'politician': politician,
    'type_of_speech': type_of_speech,
    'file_name': file_name,
    'text': text
})
df

,politician,type_of_speech,file_name,text
0,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_2025_press_releases_senator_angela_...,filter press close press filter press releases...
1,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_no_on_cr_final_p...,skip to content home about biography voting re...
2,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_no_on_cr_process...,skip to content home about biography voting re...
3,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_van_hollen_elfre...,skip to content home about biography voting re...
4,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_wyden_continue_t...,skip to content home about biography voting re...
...,...,...,...,...
1601,vargas_juan,bipartisan_and_other_speeches,bipartisan_unknown_rep_juan_vargas_ranking_mem...,
1602,vargas_juan,bipartisan_and_other_speeches,bipartisan_unknown_rep_juan_vargas_san_diego_d...,
1603,vargas_juan,bipartisan_and_other_speeches,bipartisan_unknown_rep_juan_vargas_slams_trump...,
1604,vargas_juan,bipartisan_and_other_speeches,bipartisan_unknown_rep_juan_vargas_statement_o...,


### Hugging Face Model Limitations

Only accepts 512 tokens at a time. . .

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    "bhadresh-savani/distilbert-base-uncased-emotion"
)

def chunk_text(text, max_tokens=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    # break tokens into lists of <=512
    chunked = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    # decode each chunk back to text
    return [tokenizer.decode(chunk) for chunk in chunked]

In [9]:
def classify_long_text(text, classifier, max_len=512):
    # handle NaNs. . . 
    if not isinstance(text, str) or text.strip() == "":
        return {label: 0.0 for label in ["sadness","joy","love","anger","fear","surprise"]}

    # split text into chunks
    words = text.split()
    chunks = []
    current = []

    for w in words:
        current.append(w)
        if len(current) >= max_len:
            chunks.append(" ".join(current))
            current = []

    if current:
        chunks.append(" ".join(current))

    # if no chunks created
    if len(chunks) == 0:
        return {label: 0.0 for label in ["sadness","joy","love","anger","fear","surprise"]}

    # run classifier
    results = classifier(chunks, truncation=True)

    # if classifier returned empty
    if len(results) == 0:
        return {label: 0.0 for label in ["sadness","joy","love","anger","fear","surprise"]}

    # extract labels once
    labels = [d["label"] for d in results[0]]

    # build score matrix
    score_matrix = np.array([[d["score"] for d in chunk] for chunk in results])

    # average score per label
    avg_scores = score_matrix.mean(axis=0)

    return dict(zip(labels, avg_scores))


In [10]:
# small batch test
classify_long_text(df.loc[0, "text"], classifier)

{'sadness': np.float64(0.30374593287706375),
 'joy': np.float64(0.2368980422616005),
 'love': np.float64(0.007146923337131739),
 'anger': np.float64(0.3896482530981302),
 'fear': np.float64(0.06090060668066144),
 'surprise': np.float64(0.0016602433752268553)}

In [11]:
df["sentiment_scores"] = df["text"].apply(lambda t: classify_long_text(t, classifier))

In [12]:
df.head()

,politician,type_of_speech,file_name,text,sentiment_scores
0,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_2025_press_releases_senator_angela_...,filter press close press filter press releases...,"{'sadness': 0.30374593287706375, 'joy': 0.2368..."
1,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_no_on_cr_final_p...,skip to content home about biography voting re...,"{'sadness': 0.021962977945804596, 'joy': 0.266..."
2,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_no_on_cr_process...,skip to content home about biography voting re...,"{'sadness': 0.021962977945804596, 'joy': 0.266..."
3,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_van_hollen_elfre...,skip to content home about biography voting re...,"{'sadness': 0.021962977945804596, 'joy': 0.266..."
4,alsobrooks_angela_d,bipartisan_and_other_speeches,bipartisan_unknown_alsobrooks_wyden_continue_t...,skip to content home about biography voting re...,"{'sadness': 0.021962977945804596, 'joy': 0.266..."


In [4]:
df1, df2, df3 = np.array_split(df, 3)

C:\Users\erina\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
df1.to_csv("output1.csv", index=False)
df2.to_csv("output2.csv", index=False)
df3.to_csv("output3.csv", index=False)